# Introdução
Este notebook se refere ao teste técnico do processo seletivo de analista de data science para a Mosaiclab do grupo Gouvêa Ecosystem.

O desafio consiste, sumariamente, em tratar e tirar insights uma arquivo tabular do tipo .asc.

# Leitura e tratamento

In [650]:
from google.colab import drive #os arquivos apresentados foram subidos no meu drive pessoal
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [651]:
import chardet #obtendo o encoding do arquivo asc

with open('/content/drive/MyDrive/Mosaiclab_desafio/BD.asc', 'rb') as f:
    result = chardet.detect(f.read(10000))

print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [652]:
import pandas as pd

In [653]:
df = pd.read_fwf('/content/drive/MyDrive/Mosaiclab_desafio/BD.asc', colspecs='infer', encoding='ISO-8859-1', header=None) #leitura do arquivo

In [654]:
df.head()

,0,1,2,3,4,5,6,7
0,SP,ATIBAIA,2024-05-24T21:48:35-03:00,3.807669e+12,Descrição,3484.0,1.004497e+06,4.687220e+06
1,MG,IBIRITE,2024-05-10T18:10:57-03:00,1.679328e+12,Descrição,4714.0,1.004497e+06,4.687220e+06
2,MG,ITABIRA,2024-05-14T13:10:10-03:00,1.680430e+12,Descrição,2477.0,1.004497e+06,4.687220e+06
3,SP,CAMPINAS,2024-05-04T06:26:43-03:00,3.808158e+12,Descrição,3118.0,1.004497e+06,4.687220e+06
4,SP,OURINHOS,2024-05-23T17:58:19-03:00,1.679328e+12,Descrição,4714.0,1.004497e+06,4.687220e+06


A separação da coluna veio aproximadamente correta, a não ser pelas de índices 4 e 5, que deveriam ter vindo juntas.

In [655]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63943 entries, 0 to 63942
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       63943 non-null  object 
 1   1       63943 non-null  object 
 2   2       63943 non-null  object 
 3   3       61523 non-null  float64
 4   4       63941 non-null  object 
 5   5       63941 non-null  float64
 6   6       63943 non-null  float64
 7   7       63943 non-null  float64
dtypes: float64(4), object(4)
memory usage: 3.9+ MB


As colunas de índices 3, 4 e 5 são as únicas que apresentam dados nulos.

In [656]:
cean_nulos = df.iloc[:,3].isna().sum() #guardando a quantidade de nulos na coluna 3, para caso seja útil futuramente

In [657]:
cean_nulos

2420

In [658]:
df[df.isna().any(axis=1)] #linhas com nulos


,0,1,2,3,4,5,6,7
21,PE,MANARI,2024-05-04T14:41:00-03:00,NaN,Descrição,3095.0,1.004497e+06,4.687220e+06
28,TO,PALMAS,2024-05-16T18:07:18-03:00,NaN,Descrição,5569.0,1.004497e+06,4.687220e+06
36,SP,GUARUJA,2024-05-31T10:32:05-03:00,NaN,Descrição,277.0,1.004497e+06,4.687220e+06
68,SP,INDAIATUBA,2024-05-08T02:53:48-03:00,NaN,Descrição,5479.0,1.004497e+06,4.687220e+06
79,PE,MANARI,2024-05-04T14:41:00-03:00,NaN,Descrição,2167.0,1.004497e+06,4.687220e+06
...,...,...,...,...,...,...,...,...
63922,SP,SAO JOSE DOS CAMPOS,2024-05-28T18:31:17-03:00,NaN,Descrição,836.0,1.193880e+06,4.685326e+06
63924,SP,ITAQUAQUECETUBA,2024-05-06T13:37:14-03:00,NaN,Descrição,836.0,1.193880e+06,4.685326e+06
63927,SP,SAO PAULO,2024-05-09T19:14:03-03:00,NaN,Descrição,202.0,1.193880e+06,4.685326e+06
63928,SP,JACAREI,2024-05-03T03:53:38-03:00,NaN,Descrição,836.0,1.193880e+06,4.685326e+06


In [659]:
df.iloc[:,3] = df.iloc[:,3].fillna(0) #preenchendo os nulos da coluna 3 com 0

In [660]:
df[[4]][df[[4]].isna().any(axis=1)]


,4
12209,NaN
16000,NaN


In [661]:
df[[5]][df[[5]].isna().any(axis=1)]

,5
12209,NaN
16000,NaN


As linhas com nulos da coluna 4 e 5 são as mesmas, o que é o esperado.

In [662]:
df[[5]][df[[5]].eq('0').any(axis=1)] #verificando a possibilidade de preencher os nulos da coluna 5 com 0

,5


### Etapa 2 - Item 1
Filtro: Remover da base de dados todas as linhas de descrições (DESC) que terminem com números ímpares (por exemplo, Descrição 3481), deixando apenas as descrições que terminem com números pares (por exemplo, Descrição 3118).

In [663]:
import numpy as np

In [664]:
col5 = df.iloc[:,5].fillna(0).astype(np.int32)

In [665]:
df[col5 % 2 == 0].head()

,0,1,2,3,4,5,6,7
0,SP,ATIBAIA,2024-05-24T21:48:35-03:00,3.807669e+12,Descrição,3484.0,1.004497e+06,4.687220e+06
1,MG,IBIRITE,2024-05-10T18:10:57-03:00,1.679328e+12,Descrição,4714.0,1.004497e+06,4.687220e+06
3,SP,CAMPINAS,2024-05-04T06:26:43-03:00,3.808158e+12,Descrição,3118.0,1.004497e+06,4.687220e+06
4,SP,OURINHOS,2024-05-23T17:58:19-03:00,1.679328e+12,Descrição,4714.0,1.004497e+06,4.687220e+06
5,SP,TAUBATE,2024-05-06T19:31:34-03:00,1.679328e+12,Descrição,3176.0,1.004497e+06,4.687220e+06


In [666]:
df[col5 % 2 == 0].shape

(34350, 8)

In [667]:
df = df[col5 % 2 == 0] #filtrando apenas as descrições de valor par

### De volta ao tratamento

In [668]:
aux = df.iloc[:,4].fillna('Descrição') + " " + df.iloc[:,5].fillna(0).astype(np.int32).astype(str) #concatenando a coluna de descrição, como deveria ser

In [669]:
aux.head()

,0
0,Descrição 3484
1,Descrição 4714
3,Descrição 3118
4,Descrição 4714
5,Descrição 3176


In [670]:
aux.isna().sum()

0

Agora o dataframe não possui mais células do tipo NaN.

In [671]:
aux.shape

(34350,)

In [672]:
df.iloc[:,4] = aux

In [673]:
df.iloc[:,4].head()

,4
0,Descrição 3484
1,Descrição 4714
3,Descrição 3118
4,Descrição 4714
5,Descrição 3176


In [674]:
df.drop(columns=[5], inplace=True) #eliminando a coluna que não é mais necessária

<ipython-input-674-aefa9971b129>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=[5], inplace=True) #eliminando a coluna que não é mais necessária


In [675]:
df.head()

,0,1,2,3,4,6,7
0,SP,ATIBAIA,2024-05-24T21:48:35-03:00,3.807669e+12,Descrição 3484,1.004497e+06,4.687220e+06
1,MG,IBIRITE,2024-05-10T18:10:57-03:00,1.679328e+12,Descrição 4714,1.004497e+06,4.687220e+06
3,SP,CAMPINAS,2024-05-04T06:26:43-03:00,3.808158e+12,Descrição 3118,1.004497e+06,4.687220e+06
4,SP,OURINHOS,2024-05-23T17:58:19-03:00,1.679328e+12,Descrição 4714,1.004497e+06,4.687220e+06
5,SP,TAUBATE,2024-05-06T19:31:34-03:00,1.679328e+12,Descrição 3176,1.004497e+06,4.687220e+06


In [676]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34350 entries, 0 to 63942
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       34350 non-null  object 
 1   1       34350 non-null  object 
 2   2       34350 non-null  object 
 3   3       34350 non-null  float64
 4   4       34350 non-null  object 
 5   6       34350 non-null  float64
 6   7       34350 non-null  float64
dtypes: float64(3), object(4)
memory usage: 3.1+ MB


Tudo certo!

In [677]:
with open('/content/drive/MyDrive/Mosaiclab_desafio/3BD(Dificuldade alta).flat', 'r') as f: #obtendo o nome das colunas a partir do arquivo flat
    nome_col = [linha.split('"')[1] for linha in f if linha.startswith('field')]

print(nome_col)

['UF', 'City', 'emission', 'CEAN', 'DESC', 'CNAE', 'NCM']


In [678]:
df.columns = nome_col #substituindo os índices pelos nomes das colunas

In [679]:
df.columns

Index(['UF', 'City', 'emission', 'CEAN', 'DESC', 'CNAE', 'NCM'], dtype='object')

In [680]:
df.head()

,UF,City,emission,CEAN,DESC,CNAE,NCM
0,SP,ATIBAIA,2024-05-24T21:48:35-03:00,3.807669e+12,Descrição 3484,1.004497e+06,4.687220e+06
1,MG,IBIRITE,2024-05-10T18:10:57-03:00,1.679328e+12,Descrição 4714,1.004497e+06,4.687220e+06
3,SP,CAMPINAS,2024-05-04T06:26:43-03:00,3.808158e+12,Descrição 3118,1.004497e+06,4.687220e+06
4,SP,OURINHOS,2024-05-23T17:58:19-03:00,1.679328e+12,Descrição 4714,1.004497e+06,4.687220e+06
5,SP,TAUBATE,2024-05-06T19:31:34-03:00,1.679328e+12,Descrição 3176,1.004497e+06,4.687220e+06


In [681]:
colunas_float = df.select_dtypes(include=['float64']).columns #variáveis do tipo float: CEAN, CNAE e NCM

In [682]:
colunas_float

Index(['CEAN', 'CNAE', 'NCM'], dtype='object')

In [683]:
df[colunas_float].apply(lambda x: x.astype(str)) #convertendo as variáveis do tipo float para string

,CEAN,CNAE,NCM
0,3807669171176.36,1004496.78723404,4687219.76595745
1,1679327831602.96,1004496.78723404,4687219.76595745
3,3808157751407.0,1004496.78723404,4687219.76595745
4,1679327831602.96,1004496.78723404,4687219.76595745
5,1679327831589.98,1004496.78723404,4687219.76595745
...,...,...,...
63936,1679147021462.32,1193879.9787234,4685326.14893617
63937,1679765963201.26,1193879.9787234,4685326.14893617
63939,1679765963201.26,1193879.9787234,4685326.14893617
63941,1679766106722.32,1193879.9787234,4685326.14893617


In [684]:
df[colunas_float] = df[colunas_float].apply(lambda x: x.astype(str)) #convertendo as variáveis do tipo float para string

In [685]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34350 entries, 0 to 63942
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   UF        34350 non-null  object
 1   City      34350 non-null  object
 2   emission  34350 non-null  object
 3   CEAN      34350 non-null  object
 4   DESC      34350 non-null  object
 5   CNAE      34350 non-null  object
 6   NCM       34350 non-null  object
dtypes: object(7)
memory usage: 3.1+ MB


# Etapa 1
Você deve criar um datamap (arquivo .flat) com as seguintes informações: nome, tamanho, tipo, início e fim de cada coluna.

In [686]:
inicio = 1
datamap = []

for col in df.columns:
    dtype = str(df[col].dtype) #tipo da coluna em questão
    tamanho = None
    if col == 'emission':
        tamanho = df[col].astype(str).str.len().max() #número máximo de caracteres da coluna em questão
        tipo_col = 'datetime'
    elif dtype == 'object':
        tamanho = df[col].str.len().max()
        tipo_col = 'text'
#    elif dtype in ['int64', 'float64']:
#        tamanho = df[col].astype(str).str.len().max()
#        tipo_col = 'numeric'
#    if col == 'emission':
#        tamanho = df[col].astype(str).str.len().max()
#        tipo_col = 'datetime'

    fim = inicio + tamanho - 1
    datamap.append(f'field "{col}" {tamanho} {tipo_col} {inicio} {fim}') #criando as linhas do datamap
    inicio = fim + 1

with open('datamap.flat', 'w') as f: #criando o arquivo
    f.write('\n'.join(datamap))

O arquivo datamap.flat foi criado!

# Etapa 2
- [x] Filtro: Remover da base de dados todas as linhas de descrições (DESC) que terminem com números ímpares (por exemplo, Descrição 3481), deixando apenas as descrições que terminem com números pares (por exemplo, Descrição 3118).
2. Análise-Resumo: Identificar quais são os CNAEs únicos e quantos eles são.
3. Análise-Resumo: Identificar quais os UFs únicos e quantos são.
4. Análise-Cruzamento: Indicar o produto (CEAN) de maior reincidência por estado (UF).
5. Análise-Cruzamento: Indicar o produto (CEAN) de maior reincidência por estado (UF) e por CNAE.

### Etapa 2 - Item 2
Análise-Resumo: Identificar quais são os CNAEs únicos e quantos eles são.

In [687]:
df.CNAE.unique()

array(['1004496.78723404', '1193879.55319149', '1193879.9787234'],
      dtype=object)

In [688]:
df.CNAE.nunique()

3

Há três CNAEs únicos.

### Etapa 2 - Item 3
Análise-Resumo: Identificar quais os UFs únicos e quantos são.

In [689]:
df.UF.unique()

array(['SP', 'MG', 'PR', 'RJ', 'SC', 'ES', 'RN', 'RS', 'SE', 'PA', 'MS',
       'GO', 'PE', 'PI', 'CE', 'DF', 'BA', 'AM', 'MA', 'TO', 'AL', 'RR',
       'MT', 'RO', 'PB', 'AC', 'AP'], dtype=object)

In [690]:
df.UF.nunique()

27

Há 27 unidades federativas, como era de se esperar.

### Etapa 2 - Item 4
Análise-Cruzamento: Indicar o produto (CEAN) de maior reincidência por estado (UF).
> Aqui serão considerados apenas os CEANs não nulos.

In [691]:
df_cean_valido = df[df.CEAN != '0.0'] #filtrando os CNAEs nulos

In [692]:
contagem_cean = df_cean_valido.groupby(['UF', 'CEAN']).size().reset_index(name='contagem') #agrupando e fazendo contagem de CNAEs
maior_contagem_cean = contagem_cean.loc[contagem_cean.groupby('UF')['contagem'].idxmax()].reset_index(drop=True).sort_values(by='UF') #obtendo a contagem máxima por UF e ordenando por UF
maior_contagem_cean

,UF,CEAN,contagem
0,AC,1679147021770.4,3
1,AL,1679765959597.85,7
2,AM,1680737276710.19,6
3,AP,1679147021749.34,1
4,BA,1679147021749.34,331
5,CE,1679147021749.34,166
6,DF,1679766106636.57,31
7,ES,1679147021749.34,306
8,GO,1679765961675.3,14
9,MA,1679765959597.85,36


### Etapa 2 - Item 5
Análise-Cruzamento: Indicar o produto (CEAN) de maior reincidência por estado (UF) e por CNAE.
> Aqui serão considerados apenas os CEANs não nulos.

In [693]:
contagem_cean_cnae = df_cean_valido.groupby(['UF', 'CNAE','CEAN']).size().reset_index(name='contagem') #agrupando e fazendo contagem de CNAEs
maior_contagem_cean_cnae = contagem_cean_cnae.loc[contagem_cean_cnae.groupby(['UF', 'CNAE'])['contagem'].idxmax()].reset_index(drop=True).sort_values(by='UF') #obtendo a contagem máxima por UF e por CNAE e ordenando por UF
maior_contagem_cean_cnae

,UF,CNAE,CEAN,contagem
0,AC,1193879.55319149,1679147022194.66,2
1,AC,1193879.9787234,1679147021770.4,2
2,AL,1004496.78723404,1679765959965.94,2
3,AL,1193879.55319149,5935085109589.55,4
4,AL,1193879.9787234,1679147021770.4,5
...,...,...,...,...
72,SP,1004496.78723404,3807715638308.7,219
74,SP,1193879.9787234,1679147021749.34,373
76,TO,1193879.55319149,1679765961675.3,7
75,TO,1004496.78723404,1679766081648.91,2


# Etapa 3
Converter o arquivo final, contendo todas as informações solicitadas nas etapas anteriores, para o formato .xlsx para envio.

In [694]:
df.to_excel('item1.xlsx', index=False) #exportando a base sem descrições ímpares para xlsx
maior_contagem_cean.to_excel('item4.xlsx', index=False) #exportando o agrupamento por UF
maior_contagem_cean_cnae.to_excel('item5.xlsx', index=False) #exportando o agrupamento por UF e por CNAE

Pronto! Arquivos exportados.

# Conclusão
A partir de agora, as manipulações serão feitas no próprio excel, o qual permite uma melhor visualização.